In [ ]:
# Final Token Scoring Engine (Python)
# ------------------------------------------------
# This script scores Ethereum-based tokens using a combination of
# on-chain, off-chain, and social metrics.
# ------------------------------------------------
#!pip install web3

import requests
from web3 import Web3
import json


In [ ]:
#!pip install scikit-learn

In [ ]:

# ------------------ ML IMPORTS ------------------ #
from sklearn.linear_model import LinearRegression
import numpy as np


In [ ]:
# ------------------ CONFIG ------------------ #
INFURA_URL = "https://mainnet.infura.io/v3/4541a90c12644bad8a3f5ef811f79204"
ETHERSCAN_API_KEY = "YIV1RV1ZF983R4HC7PF5FBTYFHIHWQQT9V"
DUNE_API_KEY = "zHqyJwJnGmoIn1qNCZ82qZ5XiNKN6iYw"

In [ ]:
# Connect to Ethereum node
web3 = Web3(Web3.HTTPProvider(INFURA_URL))
print("Connected to Ethereum:", web3.isConnected())

In [ ]:
# ------------------ UTILS ------------------ #
def get_token_info(address):
    """Get basic token info using ERC20 ABI."""
    erc20_abi = [
        {"constant": True, "inputs": [], "name": "name", "outputs": [{"name": "", "type": "string"}], "type": "function"},
        {"constant": True, "inputs": [], "name": "symbol", "outputs": [{"name": "", "type": "string"}], "type": "function"},
        {"constant": True, "inputs": [], "name": "totalSupply", "outputs": [{"name": "", "type": "uint256"}], "type": "function"}
    ]
    contract = web3.eth.contract(address=web3.toChecksumAddress(address), abi=erc20_abi)
    name = contract.functions.name().call()
    symbol = contract.functions.symbol().call()
    supply = contract.functions.totalSupply().call()
    return name, symbol, supply


In [ ]:
# ------------------ ETHERSCAN ------------------ #
def get_top_holders(token_address):
    """Fetch top holders (requires custom analysis or paid services, simplified here)."""
    # You could use TokenSniffer or manual scraping in free versions
    return 0.3  # Placeholder: e.g., 30% held by top 5 holders

def is_contract_verified(token_address):
    url = f"https://api.etherscan.io/api?module=contract&action=getabi&address={token_address}&apikey={ETHERSCAN_API_KEY}"
    r = requests.get(url).json()
    return r['status'] == '1'


In [ ]:

# ------------------ ML TRAINING ------------------ #
# Example training dataset: [wallet_growth, verified (0/1), top_holder_ratio, liquidity]
X_train = np.array([
    [800, 1, 0.3, 50000],
    [1000, 1, 0.2, 150000],
    [500, 0, 0.5, 10000],
    [1500, 1, 0.1, 200000]
])
# Target success scores (e.g., price % growth or internal score)
y_train = np.array([30, 60, 10, 90])

# Train linear regression model
ml_model = LinearRegression()
ml_model.fit(X_train, y_train)


In [ ]:
# ------------------ DUNE API ------------------ #
def get_wallet_growth_dune(dune_query_id):
    """Fetch wallet growth data from Dune query."""
    url = f"https://api.dune.com/api/v1/query/{dune_query_id}/results"
    headers = {"x-dune-api-key": DUNE_API_KEY}
    r = requests.get(url, headers=headers)
    if r.status_code == 200:
        data = r.json()
        # Parse your wallet growth logic
        return 800  # Placeholder: 800 new wallets in 7d
    else:
        return 0


In [ ]:

# ------------------ MAIN ------------------ #
if __name__ == "__main__":
    user_address = input("Enter the token address (0x...): ").strip()
    token_name = input("Enter the token name (e.g., Shiba Inu): ").strip()
    model_choice = input("Use Machine Learning model? (yes/no): ").strip().lower()

    name, symbol, supply = get_token_info(user_address)
    print(f"Analyzing Token: {name} ({symbol}) - Total Supply: {supply}")

    if model_choice == "yes":
        score_token_with_ml(user_address, token_name)
    else:
        score_token(user_address, token_name)


In [ ]:
# ------------------ UNISWAP SUBGRAPH ------------------ #
def get_liquidity_from_uniswap(token_address):
    """Pull liquidity data from Uniswap Subgraph."""
    query = {
        "query": """
        {
          token(id: "%s") {
            derivedETH
            totalLiquidity
          }
        }
        """ % token_address.lower()
    }
    try:
        response = requests.post(
            "https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2",
            json=query,
            headers={'Content-Type': 'application/json'}
        )
        
        if response.status_code == 200:
            data = response.json()
            if 'data' in data and 'token' in data['data'] and data['data']['token'] is not None:
                return float(data['data']['token']['totalLiquidity'])
            else:
                print(f"Token {token_address} not found on Uniswap V2")
                return 0
        else:
            print(f"Error fetching Uniswap data: {response.status_code}")
            return 0
    except Exception as e:
        print(f"Exception while fetching Uniswap data: {str(e)}")
        return 0

In [ ]:
# ------------------ SCORING ------------------ #
def score_token(token_address):
    score = 0

    # On-chain Momentum
    wallet_growth = get_wallet_growth_dune("DUNE_QUERY_ID")
    on_chain_score = min(wallet_growth / 1000, 1.0) * 10

    # Contract Verified
    verified = is_contract_verified(token_address)
    credibility_score = 2 if verified else 0

    # Top Holder Ratio (lower is better)
    top_holders_ratio = get_top_holders(token_address)
    tokenomics_score = (1 - top_holders_ratio) * 5

    # Liquidity from Uniswap
    liquidity = get_liquidity_from_uniswap(token_address)
    liquidity_score = min(liquidity / 100000, 1.0) * 10

    score = on_chain_score + credibility_score + tokenomics_score + liquidity_score

    print("\n--- Token Score Breakdown ---")
    print("Wallet Growth Score:", round(on_chain_score, 2))
    print("Credibility Score:", round(credibility_score, 2))
    print("Holder Concentration Score:", round(tokenomics_score, 2))
    print("Liquidity Score:", round(liquidity_score, 2))
    print("Total Score:", round(score, 2), "/ 100")


In [ ]:
# ------------------ MAIN ------------------ #
if __name__ == "__main__":
    user_input = input("Enter the token address (0x...): ")
    token_address = user_input.strip()
    name, symbol, supply = get_token_info(token_address)
    print(f"Analyzing Token: {name} ({symbol}) - Total Supply: {supply}")
    score_token(token_address)
